In [ ]:
from keras.models import load_model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from nltk.tokenize import word_tokenize
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('wordnet')
!pip install -q "nltk==3.4.5"
from google.colab import files
import nltk
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
from nltk.lm import Vocabulary
import gensim
from gensim.matutils import softcossim 
from gensim import corpora
import gensim.downloader as api
from gensim.utils import simple_preprocess

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
model_ali = load_model('ali wong.h5')
model_kenny=load_model('kenny sebastian.h5')

In [ ]:
def perplexity(self, text):
  return pow(2.0, self.entropy(text))

In [ ]:
data = pd.read_pickle("Cleaned_Transcript.pickle")
data.head()

,Artist,Title,Transcript,Cleaned_Transcript,Tokens,Cleaned_Artist
0,Chris Rock Total Blackout: The Tamborine Exten...,Chris Rock Total Blackout: The Tamborine Exten...,"[Jimmy Fallon] Were you at the, uh, White Hous...",were you at the uh white house party yes i was...,"[uh, white, house, party, yes, white, house, e...",chris rock
1,"Bo Burnham: Words, Words, Words (2010) – Trans...","Bo Burnham: Words, Words, Words (2010) – Trans...",(Cheers and applause)\nThank you.\n(Laughter)\...,thank you when i say hey you say ho hey ho hey...,"[thank, say, hey, say, ho, hey, ho, hey, ho, b...",bo burnham
2,Vir Das: Outside in – The Lockdown Special (20...,Vir Das: Outside in – The Lockdown Special (20...,[soft piano music playing]\n[Vir Das] What you...,what you are about to watch was not supposed t...,"[watch, supposed, happen, completely, unscript...",vir das
3,Larry the Cable Guy – Remain Seated (2020) – T...,Larry the Cable Guy – Remain Seated (2020) – T...,"[Announcer] Ladies and gentlemen, Larry, The C...",ladies and gentlemen larry the cable guy all r...,"[lady, gentleman, larry, cable, guy, right, th...",larry the cable guy
4,Craig Ferguson: Just Being Honest (2015) – Tra...,Craig Ferguson: Just Being Honest (2015) – Tra...,Watch the full show for free on YouTube\n[bagp...,watch the full show for free on youtube its a ...,"[watch, full, show, free, youtube, great, day,...",craig ferguson


In [ ]:
np.random.seed(32)
data = pd.read_pickle("Cleaned_Transcript.pickle")
data = data[['Cleaned_Artist', 'Cleaned_Transcript', 'Tokens']]
data.columns = ['Artist', 'Transcript', 'Tokens']

artist = 'kenny sebastian'
data = data[artist == data['Artist']].reset_index().drop("index", axis=1)
data_list = []
# with stop words
stop_words = True
if stop_words:
    lemmatizer = WordNetLemmatizer()
    data['Tokens'] = data['Transcript'].apply(lambda x : [lemmatizer.lemmatize(w) for w in word_tokenize(x.lower())])

data.iloc[:, 2].apply(lambda x : data_list.extend(x))
print("Length of Data List: ", len(data_list))
data = data_list
del data_list

n_words = len(data)
unique_words = len(set(data))
print('Total Words: %d' % n_words)
print('Unique Words: %d\n' % unique_words)

length = 100 + 1
lines = []
for i in range(length, n_words):
    seq = data[i - length : i]
    line = ' '.join(seq)
    lines.append(line)

Length of Data List:  178628
Total Words: 178628
Unique Words: 3632



In [ ]:
def generate_texts(model, tokenizer, seed_text, n_words, seq_length):
        print("_"*150)
        text = []
        print(seed_text, end='\n\n')
        for _ in range(n_words):
            encoded = tokenizer.texts_to_sequences([seed_text])[0]
            encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
            y_predict = np.argmax(model.predict(encoded), axis=-1)

            predicted_word = ''
            for word, index in tokenizer.word_index.items():
                if index == y_predict:
                    predicted_word = word
                    break
            seed_text = seed_text + ' ' + predicted_word
            text.append(predicted_word)
        seed_text_tokenized=nltk.tokenize.word_tokenize(seed_text)
        return (text,seed_text_tokenized)

In [ ]:
index = 0
def test_model(model, lines, input_length=100, stop_words=False):
    global index
    if index == 0:
        index = np.random.randint(0, len(lines))

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    sequences = tokenizer.texts_to_sequences(lines)
    sequences = np.array(sequences)

    vocab_size = len(tokenizer.word_index) + 1
    word_2_index = tokenizer.word_index

    X, y = sequences[:, :-1], sequences[:, -1]
    y = to_categorical(y, vocab_size)

    seq_length = X.shape[1]
    print("X shape:", X.shape)
    print("y shape:", y.shape)


    return generate_texts(model, tokenizer, lines[index], 50, seq_length)

In [ ]:
sent=test_model(model_kenny, lines, input_length=100, stop_words=stop_words)

X shape: (178527, 100)
y shape: (178527, 3633)
______________________________________________________________________________________________________________________________________________________
you put them in a group they will be like just they are the worst people like woman who are dating guy you know what i am talking about your boyfriend one on one with you is super emotional they will be like baby i just want to say you mean the world to me baby can you hold my hand while i talk to you can you hold my hand it is almost like the past eight month have become the best eight month can you do that finger thing can we do that finger thing i just want to



In [ ]:
print(sent[0])
print('..........................')
print(sent[1])

['animal', 'sort', 'a', 'admitting', 'in', 'where', 'have', 'insane', 'there', 'can', 'you', 'evil', 'then', 'way', 'want', 'dad', 'restaurant', 'shaving', 'cop', 'late', 'too', 'is', 'dad', 'boil', 'you', 'on', 'a', 'reach', 'left', 'cuz', '26', 'that', 'man', 'initiative', 'caught', 'do', 'it', 'long', 'like', 'bad', 'ha', 'gold', 'bluff', 'so', 'no', 'it', 'to', 'me', 'they', 'boil']
..........................
['you', 'put', 'them', 'in', 'a', 'group', 'they', 'will', 'be', 'like', 'just', 'they', 'are', 'the', 'worst', 'people', 'like', 'woman', 'who', 'are', 'dating', 'guy', 'you', 'know', 'what', 'i', 'am', 'talking', 'about', 'your', 'boyfriend', 'one', 'on', 'one', 'with', 'you', 'is', 'super', 'emotional', 'they', 'will', 'be', 'like', 'baby', 'i', 'just', 'want', 'to', 'say', 'you', 'mean', 'the', 'world', 'to', 'me', 'baby', 'can', 'you', 'hold', 'my', 'hand', 'while', 'i', 'talk', 'to', 'you', 'can', 'you', 'hold', 'my', 'hand', 'it', 'is', 'almost', 'like', 'the', 'past', 

In [ ]:
def unigram_test(sent):
  train_sentences = sent[1]
  tokenized_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) 
                  for sent in train_sentences]
  n = 1
  train_data, padded_vocab = padded_everygram_pipeline(n, tokenized_text)
  model = MLE(n)
  model.fit(train_data, padded_vocab)

  test_sentences = sent[0]
  tokenized_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) 
                  for sent in test_sentences]

  test_data, _ = padded_everygram_pipeline(n, tokenized_text)
  for test in test_data:
      print ("MLE Estimates:", [((ngram[-1], ngram[:-1]),model.score(ngram[-1], ngram[:-1])) for ngram in test])

  test_data, _ = padded_everygram_pipeline(n, tokenized_text)

  for i, test in enumerate(test_data):
    print("PP({0}):{1}".format(test_sentences[i], model.perplexity(test)))

In [ ]:
def bigram_test(sent):
  train_sentences = sent[1]
  tokenized_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) for sent in train_sentences]

  n = 2
  train_data = [nltk.bigrams(t,  pad_right=True, pad_left=True, left_pad_symbol="<s>", right_pad_symbol="</s>") for t in tokenized_text]
  words = [word for sent in tokenized_text for word in sent]
  words.extend(["<s>", "</s>"])
  padded_vocab = Vocabulary(words)
  model = MLE(n)
  model.fit(train_data, padded_vocab)

  test_sentences = sent[0]
  tokenized_text = [list(map(str.lower, nltk.tokenize.word_tokenize(sent))) for sent in test_sentences]

  test_data = [nltk.bigrams(t,  pad_right=True, pad_left=True, left_pad_symbol="<s>", right_pad_symbol="</s>") for t in tokenized_text]
  for test in test_data:
      print ("MLE Estimates:", [((ngram[-1], ngram[:-1]),model.score(ngram[-1], ngram[:-1])) for ngram in test])

  test_data = [nltk.bigrams(t,  pad_right=True, pad_left=True, left_pad_symbol="<s>", right_pad_symbol="</s>") for t in tokenized_text]
  for i, test in enumerate(test_data):
    print("PP({0}):{1}".format(test_sentences[i], model.perplexity(test)))

In [ ]:
unigram_test(sent)

MLE Estimates: [(('animal', ()), 0.006622516556291391)]
MLE Estimates: [(('sort', ()), 0.006622516556291391)]
MLE Estimates: [(('a', ()), 0.019867549668874173)]
MLE Estimates: [(('admitting', ()), 0.006622516556291391)]
MLE Estimates: [(('in', ()), 0.013245033112582781)]
MLE Estimates: [(('where', ()), 0.006622516556291391)]
MLE Estimates: [(('have', ()), 0.013245033112582781)]
MLE Estimates: [(('insane', ()), 0.006622516556291391)]
MLE Estimates: [(('there', ()), 0.006622516556291391)]
MLE Estimates: [(('can', ()), 0.033112582781456956)]
MLE Estimates: [(('you', ()), 0.06622516556291391)]
MLE Estimates: [(('evil', ()), 0.006622516556291391)]
MLE Estimates: [(('then', ()), 0.006622516556291391)]
MLE Estimates: [(('way', ()), 0.006622516556291391)]
MLE Estimates: [(('want', ()), 0.019867549668874173)]
MLE Estimates: [(('dad', ()), 0.013245033112582781)]
MLE Estimates: [(('restaurant', ()), 0.006622516556291391)]
MLE Estimates: [(('shaving', ()), 0.006622516556291391)]
MLE Estimates: [((

In [ ]:
bigram_test(sent)

MLE Estimates: [(('animal', ('<s>',)), 0.006622516556291391), (('</s>', ('animal',)), 1.0)]
MLE Estimates: [(('sort', ('<s>',)), 0.006622516556291391), (('</s>', ('sort',)), 1.0)]
MLE Estimates: [(('a', ('<s>',)), 0.019867549668874173), (('</s>', ('a',)), 1.0)]
MLE Estimates: [(('admitting', ('<s>',)), 0.006622516556291391), (('</s>', ('admitting',)), 1.0)]
MLE Estimates: [(('in', ('<s>',)), 0.013245033112582781), (('</s>', ('in',)), 1.0)]
MLE Estimates: [(('where', ('<s>',)), 0.006622516556291391), (('</s>', ('where',)), 1.0)]
MLE Estimates: [(('have', ('<s>',)), 0.013245033112582781), (('</s>', ('have',)), 1.0)]
MLE Estimates: [(('insane', ('<s>',)), 0.006622516556291391), (('</s>', ('insane',)), 1.0)]
MLE Estimates: [(('there', ('<s>',)), 0.006622516556291391), (('</s>', ('there',)), 1.0)]
MLE Estimates: [(('can', ('<s>',)), 0.033112582781456956), (('</s>', ('can',)), 1.0)]
MLE Estimates: [(('you', ('<s>',)), 0.06622516556291391), (('</s>', ('you',)), 1.0)]
MLE Estimates: [(('evil',

In [ ]:
model_stop_words = load_model('kenny sebastian_stop_word.h5')
model_dropout = load_model('kenny sebastian_stop_word_dropout.h5')

In [ ]:
sent_stopwords=test_model(model_stop_words, lines, input_length=100, stop_words=stop_words)

X shape: (178527, 100)
y shape: (178527, 3633)
______________________________________________________________________________________________________________________________________________________
you put them in a group they will be like just they are the worst people like woman who are dating guy you know what i am talking about your boyfriend one on one with you is super emotional they will be like baby i just want to say you mean the world to me baby can you hold my hand while i talk to you can you hold my hand it is almost like the past eight month have become the best eight month can you do that finger thing can we do that finger thing i just want to



In [ ]:
unigram_test(sent_stopwords)

MLE Estimates: [(('say', ()), 0.013245033112582781)]
MLE Estimates: [(('that', ()), 0.019867549668874173)]
MLE Estimates: [(('my', ()), 0.019867549668874173)]
MLE Estimates: [(('friend', ()), 0.013245033112582781)]
MLE Estimates: [(('like', ()), 0.052980132450331126)]
MLE Estimates: [(('me', ()), 0.026490066225165563)]
MLE Estimates: [(('and', ()), 0.006622516556291391)]
MLE Estimates: [(('gopal', ()), 0.006622516556291391)]
MLE Estimates: [(('have', ()), 0.013245033112582781)]
MLE Estimates: [(('been', ()), 0.006622516556291391)]
MLE Estimates: [(('friend', ()), 0.013245033112582781)]
MLE Estimates: [(('for', ()), 0.006622516556291391)]
MLE Estimates: [(('like', ()), 0.052980132450331126)]
MLE Estimates: [(('eight', ()), 0.026490066225165563)]
MLE Estimates: [(('year', ()), 0.006622516556291391)]
MLE Estimates: [(('but', ()), 0.006622516556291391)]
MLE Estimates: [(('your', ()), 0.013245033112582781)]
MLE Estimates: [(('eight', ()), 0.026490066225165563)]
MLE Estimates: [(('month', ()

In [ ]:
bigram_test(sent_stopwords)

MLE Estimates: [(('say', ('<s>',)), 0.013245033112582781), (('</s>', ('say',)), 1.0)]
MLE Estimates: [(('that', ('<s>',)), 0.019867549668874173), (('</s>', ('that',)), 1.0)]
MLE Estimates: [(('my', ('<s>',)), 0.019867549668874173), (('</s>', ('my',)), 1.0)]
MLE Estimates: [(('friend', ('<s>',)), 0.013245033112582781), (('</s>', ('friend',)), 1.0)]
MLE Estimates: [(('like', ('<s>',)), 0.052980132450331126), (('</s>', ('like',)), 1.0)]
MLE Estimates: [(('me', ('<s>',)), 0.026490066225165563), (('</s>', ('me',)), 1.0)]
MLE Estimates: [(('and', ('<s>',)), 0.006622516556291391), (('</s>', ('and',)), 1.0)]
MLE Estimates: [(('gopal', ('<s>',)), 0.006622516556291391), (('</s>', ('gopal',)), 1.0)]
MLE Estimates: [(('have', ('<s>',)), 0.013245033112582781), (('</s>', ('have',)), 1.0)]
MLE Estimates: [(('been', ('<s>',)), 0.006622516556291391), (('</s>', ('been',)), 1.0)]
MLE Estimates: [(('friend', ('<s>',)), 0.013245033112582781), (('</s>', ('friend',)), 1.0)]
MLE Estimates: [(('for', ('<s>',))

In [ ]:
sent_dropout=test_model(model_dropout, lines, input_length=100, stop_words=stop_words)

X shape: (178527, 100)
y shape: (178527, 3633)
______________________________________________________________________________________________________________________________________________________
you put them in a group they will be like just they are the worst people like woman who are dating guy you know what i am talking about your boyfriend one on one with you is super emotional they will be like baby i just want to say you mean the world to me baby can you hold my hand while i talk to you can you hold my hand it is almost like the past eight month have become the best eight month can you do that finger thing can we do that finger thing i just want to



In [ ]:
unigram_test(sent_dropout)

MLE Estimates: [(('say', ()), 0.013245033112582781)]
MLE Estimates: [(('that', ()), 0.019867549668874173)]
MLE Estimates: [(('my', ()), 0.019867549668874173)]
MLE Estimates: [(('friend', ()), 0.006622516556291391)]
MLE Estimates: [(('is', ()), 0.026490066225165563)]
MLE Estimates: [(('me', ()), 0.013245033112582781)]
MLE Estimates: [(('and', ()), 0.006622516556291391)]
MLE Estimates: [(('then', ()), 0.006622516556291391)]
MLE Estimates: [(('vegetarian', ()), 0.006622516556291391)]
MLE Estimates: [(('version', ()), 0.006622516556291391)]
MLE Estimates: [(('of', ()), 0.006622516556291391)]
MLE Estimates: [(('this', ()), 0.006622516556291391)]
MLE Estimates: [(('is', ()), 0.026490066225165563)]
MLE Estimates: [(('cottage', ()), 0.026490066225165563)]
MLE Estimates: [(('cheese', ()), 0.019867549668874173)]
MLE Estimates: [(('no', ()), 0.019867549668874173)]
MLE Estimates: [(('one', ()), 0.033112582781456956)]
MLE Estimates: [(('give', ()), 0.019867549668874173)]
MLE Estimates: [(('you', ()

In [ ]:
bigram_test(sent_dropout)

MLE Estimates: [(('say', ('<s>',)), 0.013245033112582781), (('</s>', ('say',)), 1.0)]
MLE Estimates: [(('that', ('<s>',)), 0.019867549668874173), (('</s>', ('that',)), 1.0)]
MLE Estimates: [(('my', ('<s>',)), 0.019867549668874173), (('</s>', ('my',)), 1.0)]
MLE Estimates: [(('friend', ('<s>',)), 0.006622516556291391), (('</s>', ('friend',)), 1.0)]
MLE Estimates: [(('is', ('<s>',)), 0.026490066225165563), (('</s>', ('is',)), 1.0)]
MLE Estimates: [(('me', ('<s>',)), 0.013245033112582781), (('</s>', ('me',)), 1.0)]
MLE Estimates: [(('and', ('<s>',)), 0.006622516556291391), (('</s>', ('and',)), 1.0)]
MLE Estimates: [(('then', ('<s>',)), 0.006622516556291391), (('</s>', ('then',)), 1.0)]
MLE Estimates: [(('vegetarian', ('<s>',)), 0.006622516556291391), (('</s>', ('vegetarian',)), 1.0)]
MLE Estimates: [(('version', ('<s>',)), 0.006622516556291391), (('</s>', ('version',)), 1.0)]
MLE Estimates: [(('of', ('<s>',)), 0.006622516556291391), (('</s>', ('of',)), 1.0)]
MLE Estimates: [(('this', ('<s

In [ ]:


import itertools

#supporting function
def _split_into_words(sentences):
  """Splits multiple sentences into words and flattens the result"""
  return list(itertools.chain(*[_.split(" ") for _ in sentences]))

#supporting function
def _get_word_ngrams(n, sentences):
  """Calculates word n-grams for multiple sentences.
  """
  assert len(sentences) > 0
  assert n > 0

  words = _split_into_words(sentences)
  return _get_ngrams(n, words)

#supporting function
def _get_ngrams(n, text):
  """Calcualtes n-grams.
  Args:
    n: which n-grams to calculate
    text: An array of tokens
  Returns:
    A set of n-grams
  """
  ngram_set = set()
  text_length = len(text)
  max_index_ngram_start = text_length - n
  for i in range(max_index_ngram_start + 1):
    ngram_set.add(tuple(text[i:i + n]))
  return ngram_set

def rouge_n(reference_sentences, evaluated_sentences, n=2):
  """
  Computes ROUGE-N of two text collections of sentences.
  
  Args:
    evaluated_sentences: The sentences that have been picked by the summarizer
    reference_sentences: The sentences from the referene set
    n: Size of ngram.  Defaults to 2.
  Returns:
    recall rouge score(float)
  Raises:
    ValueError: raises exception if a param has len <= 0
  """
  if len(evaluated_sentences) <= 0 or len(reference_sentences) <= 0:
    raise ValueError("Collections must contain at least 1 sentence.")

  evaluated_ngrams = _get_word_ngrams(n, evaluated_sentences)
  reference_ngrams = _get_word_ngrams(n, reference_sentences)
  reference_count = len(reference_ngrams)
  evaluated_count = len(evaluated_ngrams)

  # Gets the overlapping ngrams between evaluated and reference
  overlapping_ngrams = evaluated_ngrams.intersection(reference_ngrams)
  overlapping_count = len(overlapping_ngrams)

  # Handle edge case. This isn't mathematically correct, but it's good enough
  if evaluated_count == 0:
    precision = 0.0
  else:
    precision = overlapping_count / evaluated_count

  if reference_count == 0:
    recall = 0.0
  else:
    recall = overlapping_count / reference_count

  f1_score = 2.0 * ((precision * recall) / (precision + recall + 1e-8))

  #just returning recall count in rouge, useful for our purpose
  return recall

In [ ]:
ref=sent[1]
gen=sent[0]
d={}
d['sentence'] = rouge_n(ref, gen, n=2)
rouge_score
ref=sent_stopwords[1]
gen=sent_stopwords[0]
d['sentence_with_stopwords'] = rouge_n(ref, gen, n=2)
ref=sent_dropout[1]
gen=sent_dropout[0]
d['sentence_with_dropout'] = rouge_n(ref, gen, n=2)

In [ ]:
d

{'sentence': 0.3684210526315789,
 'sentence_with_dropout': 0.22935779816513763,
 'sentence_with_stopwords': 0.3543307086614173}

In [ ]:
def cosine_similarity(t1,t2):
  X_list = t1
  Y_list = t2

  l1 =[];l2 =[]
  X_set = {w for w in X_list} 
  Y_set = {w for w in Y_list}
  rvector = X_set.union(Y_set) 
  for w in rvector:
      if w in X_set: l1.append(1) # create a vector
      else: l1.append(0)
      if w in Y_set: l2.append(1)
      else: l2.append(0)
  c = 0
    
  # cosine formula 
  for i in range(len(rvector)):
          c+= l1[i]*l2[i]
  cosine = c / float((sum(l1)*sum(l2))**0.5)
  return cosine
  print("similarity: ", cosine)

In [ ]:
cosine_sentence=cosine_similarity(sent[1],sent[0])
cosine_stopwords=cosine_similarity(sent_stopwords[1],sent_stopwords[0])
cosine_dropout=cosine_similarity(sent_dropout[1],sent_dropout[0])
print("Sentence similarity: ", cosine_sentence)
print("Sentence similarity with stopwords: ", cosine_stopwords)
print("Sentence similarity with dropout: ", cosine_dropout)

Sentence similarity:  0.6460582824697986
Sentence similarity with stopwords:  0.6460582824697986
Sentence similarity with dropout:  0.6460582824697986
